In [2]:
import pandas as pd
import sys
sys.path.append('..')

In [5]:
features = pd.read_csv('data/features.csv')

In [7]:
features.feature_text.unique().shape

(131,)

In [8]:
train_data = pd.read_csv('data/train.csv')

In [9]:
patient_notes = pd.read_csv('data/patient_notes.csv')